In [1]:
import json
import os
import time
from collections import Counter, defaultdict
import numpy as np
import random
import pandas as pd
from helpers import *
from tqdm import tqdm
from seqeval.metrics import classification_report

In [3]:
data_cache = "data/nrc_fine_json/train_ner.json"
meta_path = "norec/data/metadata.json"
sentence_pol_dataset_path = "norec_sentence/3class/train.json"

save_root = "outputs/"
tabular_savefolder = os.path.join(save_root, "tabular")
os.makedirs(tabular_savefolder, exist_ok=True)
printout_folder = os.path.join(save_root, "printouts")
os.makedirs(printout_folder, exist_ok=True)
data_cache = os.path.join(tabular_savefolder, "data_sentencewise.json")

In [4]:
# Get the metadata for document-level

with open(data_cache, encoding = "utf-8") as rf:
    dataset = json.load(rf)

with open (meta_path) as rf:
    meta = json.load(rf)

inspect_ids = list(set([s["doc_id"] for s in dataset]))

ratings = []
for doc_id in inspect_ids:
    sentences = len([s for s in dataset if s["sent_id"][:6] == doc_id])
    ratings.append({"doc_id": doc_id, "rating": str(meta[doc_id]["rating"] ), "category": meta[doc_id]["category"],
        "sentences": sentences})
doc_level = pd.DataFrame(ratings)
cat_rating = []
for  category, cat_group in doc_level.groupby("category"):
    cat_rat = {rat: len(subgroup) for rat, subgroup in cat_group.groupby("rating")}
    cat_rat["category"] = category
    cat_rating.append(cat_rat)
df_cat_rat = pd.DataFrame.from_records(cat_rating).fillna(0)


doc_level.to_csv(os.path.join(tabular_savefolder,"doc_meta_50.csv"), sep="\t", decimal=",")
doc_level.head(3)

NameError: name 'inspect_ids' is not defined

In [ ]:
with open(sentence_pol_dataset_path, encoding = "utf-8") as rf:
    sentence_pol_dataset = json.load(rf)
sentence_polarities = { e['sent_id']: e['label'] for e in sentence_pol_dataset}

In [ ]:
# Create new printout 
# Get the die pips and category for the fifty
meta_path = "data/nrc_fine_json/metadata.json"
with open (meta_path) as rf:
    meta = json.load(rf)
nes= [] # One line for each NE
#    ratings.append({"doc_id": doc_id, "rating":meta[doc_id]["rating"] , "category": meta[doc_id]["category"]})
tsa_unmatched = [] # list the tsas not matched bu any NE
for sent in dataset: # Previous cell
    doc_id = sent["sent_id"].split("-")[0]
    if not doc_id in inspect_ids:
        continue
    targets = sent["targets_resolved"].copy()
    sent_nes = [ne for ne in sent["ner_firsts"] if ne["tag"] in ["PER", "ORG"]]
    for ne in sent_nes:
        ne["tsa_pol"] = None
        ne_range = set(range(ne["start"], ne["end"]))
        for tsa_range in  targets:
            resolved_start, resolved_end = element_extremes([tsa_range])
            if ne_range.intersection(range(resolved_start, resolved_end)):
                ne["tsa_pol"] = targets.pop(tsa_range) # Maybe a bit harsh to allow only one match Yes. Must be change. But now I have annotated on the output of this fine
                break

        nes.append({"doc_id":doc_id,
            "rating":meta[doc_id]["rating"] , 
            "category": meta[doc_id]["category"] ,
            "sent_id": sent["sent_id"] ,
            'sentence_pol': sent['sentence_pol'],
            "ne_text": ne["text"] ,
            "ne_tag": ne["tag"],
            "ne_tsa": ne["tsa_pol"]
            })
    for tsa_range in targets:
        resolved_start, resolved_end = element_extremes([tsa_range])
        tsa_unmatched.append({"doc_id":doc_id,
            "rating":meta[doc_id]["rating"] , 
            "category": meta[doc_id]["category"] ,
            "sent_id": sent["sent_id"] ,
            'sentence_pol': sent['sentence_pol'],
            "tsa_word": sent["text"][resolved_start: resolved_end ], 
            "tsa_pol": targets[tsa_range]
        })

joined_df = pd.DataFrame(nes).append(pd.DataFrame(tsa_unmatched), ignore_index = True)
print(joined_df.head(5))
print(joined_df.tail(5))